### Stacking Model Development

- Objective: develop and compare stacking models using best candidates from Nested CV notebook 
- Methodology: mlxtend module

We will use the augmented dataset (which does not include the duration field)

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math

In [2]:
from mlxtend.classifier import StackingCVClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, precision_score, recall_score, average_precision_score
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.utils.fixes import signature
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.float_format', lambda x: '%.3f' % x)

RANDOM_SEED = 12

In [4]:
#Loading df1 after it has been augmented in iteration 1:
df = pd.read_pickle('../data/pickle_files/df_pickle')
#Excluding the duration variable as it cannot be used in our baseline
df = df.drop(columns = ['duration'])

In [5]:
#Checking dtypes have loaded correctly (should return empty index)
df.select_dtypes(exclude = ['float64', 'int64']).columns

y = df['y']
X = df.drop(columns=['y'])

In [6]:
scaler = preprocessing.StandardScaler().fit(X)
X_transformed = scaler.transform(X)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X_transformed), y, random_state = 4)

In [8]:
#will work with numpy arrays
y = np.array(y)
X = np.array(X_transformed)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 4)

Most Promising Models from Iteration 1 - Model Comparison Notebook:
    - Decision Tree Classifier
    - K Neighbors Classifier
    - BernoulliNB
    - Logistic Regression

We will implement a stacking algorithm that uses all three and makes a prediction based on their predictions

In [10]:
def gridSearch_clf(clf, param_grid, X_train, y_train):
    gs = GridSearchCV(clf, param_grid).fit(X_train, y_train)
    print("Best Parameters")
    print(gs.best_params_)
    return gs.best_estimator_

In [11]:
def gs_report(y_test, X_test, best_estimator):
    print(classification_report(y_test, best_estimator.predict(X_test)))
    print("Overall Accuracy Score: ")
    print(accuracy_score(y_test, best_estimator.predict(X_test)))

In [12]:
clf1 = DecisionTreeClassifier()
param_grid = {'max_depth':[5,7,9,11],
              'min_samples_split':[900, 1000, 3000],
              'max_features': [7,8,10,15],
              'max_leaf_nodes':[10,20],
              'class_weight': ['balanced']}

In [14]:
#Saving best estimator
best_clf1 = gridSearch_clf(clf1, param_grid, X_train, y_train)
gs_report(y_test,X_test, best_clf1)

Best Parameters
{'class_weight': 'balanced', 'max_depth': 7, 'max_features': 7, 'max_leaf_nodes': 10, 'min_samples_split': 1000}
             precision    recall  f1-score   support

          0       0.94      0.86      0.90      9131
          1       0.35      0.60      0.44      1166

avg / total       0.88      0.83      0.85     10297

Overall Accuracy Score: 
0.8302418180052442


In [15]:
clf2 = KNeighborsClassifier()

In [16]:
param_grid = {'n_neighbors':[5,7,9,15],
              'algorithm' : ['auto', 'ball_tree'],
              'weights': ['distance']}

In [18]:
#Saving Best Estimator
best_clf2 = gridSearch_clf(clf2, param_grid, X_train, y_train)
gs_report(y_test,X_test, best_clf2)

Best Parameters
{'algorithm': 'ball_tree', 'n_neighbors': 15, 'weights': 'distance'}
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      9131
          1       0.55      0.27      0.36      1166

avg / total       0.87      0.89      0.88     10297

Overall Accuracy Score: 
0.8919102651257648


In [19]:
clf3 = BernoulliNB()
param_grid = {'alpha':np.logspace(-2, 3, num=6, base=10)}

In [20]:
#Saving best estimator
best_clf3 = gridSearch_clf(clf3, param_grid, X_train, y_train)
gs_report(y_test,X_test, best_clf3)

Best Parameters
{'alpha': 1000.0}
             precision    recall  f1-score   support

          0       0.92      0.90      0.91      9131
          1       0.31      0.35      0.33      1166

avg / total       0.85      0.84      0.84     10297

Overall Accuracy Score: 
0.835971642225891


In [21]:
def print_cv(clfs, clf_names):
    
    print('3-fold cross validation:\n')

    for clf, label in zip([best_clf1, best_clf2, best_clf3, sclf], 
                          ['Logistic Regression', 
                           'K Neighbors Classifier', 
                           'Bernoulli Naive Bayes',
                           'StackingClassifier']):

        scores = model_selection.cross_val_score(clf, X, y, cv=3, scoring='accuracy')

        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [22]:
lr = LogisticRegression()

In [23]:
np.random.seed(3)
sclf = StackingCVClassifier(classifiers=[best_clf1, best_clf2, best_clf3], 
                            meta_classifier=lr)

In [24]:
clfs = [best_clf1, best_clf2, best_clf3, sclf]
clf_names = [i.__class__.__name__ for i in clfs]

In [25]:
print_cv(clfs, clf_names)

3-fold cross validation:

Accuracy: 0.37 (+/- 0.36) [Logistic Regression]
Accuracy: 0.48 (+/- 0.29) [K Neighbors Classifier]
Accuracy: 0.67 (+/- 0.32) [Bernoulli Naive Bayes]
Accuracy: 0.47 (+/- 0.30) [StackingClassifier]


In [30]:
param_grid = {'meta-logisticregression__C':np.logspace(-2, 3, num=6, base=10)}

In [31]:
#Saving Best Estimator
best_sclf = gridSearch_clf(sclf, param_grid, X_train, y_train)
gs_report(y_test,X_test, best_sclf)

Best Parameters
{'meta-logisticregression__C': 0.01}
             precision    recall  f1-score   support

          0       0.91      0.99      0.94      9131
          1       0.65      0.19      0.29      1166

avg / total       0.88      0.90      0.87     10297

Overall Accuracy Score: 
0.8966689326988443


In [ ]:
#Trying the same stacking classifier with class probabilities rather than class labels
np.random.seed(3)
sclf_proba = StackingCVClassifier(classifiers=[best_clf1, best_clf2, best_clf3],
                            use_probas = True,
                            meta_classifier=lr)

In [ ]:
clfs = [best_clf1, best_clf2, best_clf3, sclf_proba]
clf_names = [i.__class__.__name__ for i in clfs]

In [ ]:
print_cv(clfs, clf_names)